<a href="https://colab.research.google.com/github/Aithu-Snehith/Neural-networks-Model-optimisation/blob/master/V1_mobilenet_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly
! pip install -U tf-nightly-gpu

! pip install tensorflow-model-optimization

Requirement already up-to-date: tf-nightly-gpu in /usr/local/lib/python3.6/dist-packages (1.15.0.dev20190710)


In [0]:
%load_ext tensorboard
import tensorboard

import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os

W0710 19:01:28.964322 140356938360704 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



In [0]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, GaussianDropout, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from keras.utils.np_utils import to_categorical
from sklearn.utils import class_weight
from tensorflow.keras.layers import *
import time

In [0]:
path = 'drive/path_to_data'

batch_size = 128
num_classes = 7
epochs = 100

# input image dimensions
img_rows, img_cols = 48, 48

In [0]:
# x = np.load(path + 'snehith_fer2013_fdataX.npy')
# y = np.load(path + 'snehith_fer2013_flabels.npy')

# print(x.shape, y.shape)

# x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

# np.save(path + 'x_train.npy', x_train)
# np.save(path + 'x_test.npy', x_test)
# np.save(path + 'y_train.npy', y_train)
# np.save(path + 'y_test.npy', y_test)

(35887, 48, 48, 1) (35887, 7)


In [0]:
x_train = np.load(path + 'x_train.npy')
x_test = np.load(path + 'x_test.npy')
y_train = np.load(path + 'y_train.npy')
y_test = np.load(path + 'y_test.npy')

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(32298, 48, 48, 1) (3589, 48, 48, 1) (32298, 7) (3589, 7)


In [0]:
def depth_conv_block(model, d, k, s):
	model.add(DepthwiseConv2D((k, k), strides=(s, s), padding='same', use_bias=False))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	model.add(Conv2D(d, (1, 1), padding='same', use_bias=False))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	return model


model = Sequential()
# CONV
model.add(Conv2D(int(32), (3, 3), padding='same', strides=(2, 2), input_shape=(img_rows,img_cols, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
# DEPTHWISE CONVS
model = depth_conv_block(model, 64, 3, 1)

model = depth_conv_block(model, 128, 3, 2)
model = depth_conv_block(model, 128, 3, 1)

model = depth_conv_block(model, 256, 3, 2)
model = depth_conv_block(model, 256, 3, 1)

model = depth_conv_block(model, 512, 3, 2)
model = depth_conv_block(model, 512, 3, 1)
model = depth_conv_block(model, 512, 3, 1)
model = depth_conv_block(model, 512, 3, 1)
model = depth_conv_block(model, 512, 3, 1)
model = depth_conv_block(model, 512, 3, 1)

model = depth_conv_block(model, 1024, 3, 2)
model = depth_conv_block(model, 1024, 3, 1)
# FLATTEN
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 24, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 24, 24, 32)        0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 24, 24, 32)        288       
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        2

In [0]:
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch=0)]

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0710 19:09:28.942561 140356938360704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0710 19:09:34.727003 140356938360704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 32298 samples, validate on 3589 samples
Epoch 1/100
32298/32298 [==============================] - 20s 621us/sample - loss: 1.8026 - acc: 0.2647 - val_loss: 1.8512 - val_acc: 0.2455
Epoch 2/100
32298/32298 [==============================] - 8s 245us/sample - loss: 1.6288 - acc: 0.3560 - val_loss: 1.9116 - val_acc: 0.1638
Epoch 3/100
32298/32298 [==============================] - 8s 244us/sample - loss: 1.5049 - acc: 0.4195 - val_loss: 1.5806 - val_acc: 0.3831
Epoch 4/100
32298/32298 [==============================] - 8s 245us/sample - loss: 1.4185 - acc: 0.4553 - val_loss: 1.9088 - val_acc: 0.3795
Epoch 5/100
32298/32298 [==============================] - 8s 245us/sample - loss: 1.3302 - acc: 0.4904 - val_loss: 1.7802 - val_acc: 0.4113
Epoch 6/100
32298/32298 [==============================] - 8s 245us/sample - loss: 1.2568 - acc: 0.5271 - val_loss: 1.7393 - val_acc: 0.3959
Epoch 7/100
32298/32298 [==============================] - 8s 245us/sample - loss: 1.1885 - acc: 0.5550 

In [0]:
# Backend agnostic way to save/restore models
keras_file = path + 'raw_mobilenet_model.h5'
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Saving model to:  drive/My Drive/interns/Kpit/raw_mobilenet_model.h5


In [0]:
from tensorflow_model_optimization.sparsity import keras as sparsity

epochs = 100
num_train_samples = x_train.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print('End step: ' + str(end_step))

End step: 25300


In [0]:
pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=2000,
                                                   end_step=end_step,
                                                   frequency=100)
}


def prune_depth_conv_block(model, d, k, s):
	model.add(sparsity.prune_low_magnitude(DepthwiseConv2D((k, k), strides=(s, s), padding='same', use_bias=False), **pruning_params))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	model.add(sparsity.prune_low_magnitude(Conv2D(d, (1, 1), padding='same', use_bias=False), **pruning_params))
	model.add(BatchNormalization())
	model.add(Activation('relu'))
	return model


pr_model = Sequential()
# CONV
pr_model.add(sparsity.prune_low_magnitude(Conv2D(int(32), (3, 3), padding='same', strides=(2, 2), input_shape=(img_rows,img_cols, 1)), **pruning_params))
pr_model.add(BatchNormalization())
pr_model.add(Activation('relu'))
# DEPTHWISE CONVS
pr_model = prune_depth_conv_block(pr_model, 64, 3, 1)

pr_model = prune_depth_conv_block(pr_model, 128, 3, 2)
pr_model = prune_depth_conv_block(pr_model, 128, 3, 1)

pr_model = prune_depth_conv_block(pr_model, 256, 3, 2)
pr_model = prune_depth_conv_block(pr_model, 256, 3, 1)

pr_model = prune_depth_conv_block(pr_model, 512, 3, 2)
pr_model = prune_depth_conv_block(pr_model, 512, 3, 1)
pr_model = prune_depth_conv_block(pr_model, 512, 3, 1)
pr_model = prune_depth_conv_block(pr_model, 512, 3, 1)
pr_model = prune_depth_conv_block(pr_model, 512, 3, 1)
pr_model = prune_depth_conv_block(pr_model, 512, 3, 1)

pr_model = prune_depth_conv_block(pr_model, 1024, 3, 2)
pr_model = prune_depth_conv_block(pr_model, 1024, 3, 1)
# FLATTEN
pr_model.add(GlobalAveragePooling2D())
pr_model.add(sparsity.prune_low_magnitude(Dense(num_classes, activation='softmax'), **pruning_params))
pr_model.summary()

W0710 19:25:13.846943 140356938360704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:185: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
W0710 19:25:14.600370 140356938360704 module_wrapper.py:126] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:153: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

W0710 19:25:14.791079 140356938360704 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_schedule.py:240: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d_1 (None, 24, 24, 32)        610       
_________________________________________________________________
batch_normalization_27 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
activation_27 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
prune_low_magnitude_depthwis (None, 24, 24, 32)        289       
_________________________________________________________________
batch_normalization_28 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
activation_28 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 24, 24, 64)       

In [0]:
pr_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

# Add a pruning step callback to peg the pruning step to the optimizer's
# step. Also add a callback to add pruning summaries to tensorboard
callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

pr_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          verbose=1,
          callbacks=callbacks,
          validation_data=(x_test, y_test))

score = pr_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 32298 samples, validate on 3589 samples
Epoch 1/100
32298/32298 [==============================] - 22s 697us/sample - loss: 1.7932 - acc: 0.2644 - val_loss: 1.8703 - val_acc: 0.1702
Epoch 2/100
32298/32298 [==============================] - 12s 381us/sample - loss: 1.6248 - acc: 0.3606 - val_loss: 1.9531 - val_acc: 0.1638
Epoch 3/100
32298/32298 [==============================] - 12s 379us/sample - loss: 1.5112 - acc: 0.4161 - val_loss: 1.5685 - val_acc: 0.3831
Epoch 4/100
32298/32298 [==============================] - 12s 382us/sample - loss: 1.4238 - acc: 0.4551 - val_loss: 1.8210 - val_acc: 0.3667
Epoch 5/100
32298/32298 [==============================] - 12s 383us/sample - loss: 1.3390 - acc: 0.4902 - val_loss: 1.9022 - val_acc: 0.3937
Epoch 6/100
32298/32298 [==============================] - 12s 380us/sample - loss: 1.2524 - acc: 0.5302 - val_loss: 1.8510 - val_acc: 0.3784
Epoch 7/100
32298/32298 [==============================] - 12s 385us/sample - loss: 1.1904 - acc: 0

In [0]:
checkpoint_file = path + 'pruned_mobilenet_checkpoint.h5'
print('Saving pruned model to: ', checkpoint_file)
# saved_model() sets include_optimizer to True by default. Spelling it out here
# to highlight.
tf.keras.models.save_model(pr_model, checkpoint_file, include_optimizer=True)

with sparsity.prune_scope():
  restored_model = tf.keras.models.load_model(checkpoint_file)

restored_model.fit(x_train, y_train,
                   batch_size=batch_size,
                   epochs=10,
                   verbose=1,
                   callbacks=callbacks,
                   validation_data=(x_test, y_test))

score = restored_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saving pruned model to:  drive/My Drive/interns/Kpit/pruned_mobilenet_checkpoint.h5
Train on 32298 samples, validate on 3589 samples
Epoch 1/10
32298/32298 [==============================] - 23s 717us/sample - loss: 0.1035 - acc: 0.9633 - val_loss: 3.5311 - val_acc: 0.4773
Epoch 2/10
32298/32298 [==============================] - 11s 344us/sample - loss: 0.0891 - acc: 0.9685 - val_loss: 3.5037 - val_acc: 0.5155
Epoch 3/10
32298/32298 [==============================] - 11s 342us/sample - loss: 0.0916 - acc: 0.9658 - val_loss: 3.6746 - val_acc: 0.4990
Epoch 4/10
32298/32298 [==============================] - 11s 342us/sample - loss: 0.1041 - acc: 0.9631 - val_loss: 3.8436 - val_acc: 0.4606
Epoch 5/10
32298/32298 [==============================] - 11s 347us/sample - loss: 0.1024 - acc: 0.9642 - val_loss: 3.6854 - val_acc: 0.4804
Epoch 6/10
32298/32298 [==============================] - 11s 346us/sample - loss: 0.1027 - acc: 0.9624 - val_loss: 3.5733 - val_acc: 0.4960
Epoch 7/10
32298/3229

In [0]:
final_model = sparsity.strip_pruning(pr_model)
final_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 24, 24, 32)        320       
_________________________________________________________________
batch_normalization_27 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
activation_27 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
depthwise_conv2d_13 (Depthwi (None, 24, 24, 32)        288       
_________________________________________________________________
batch_normalization_28 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
activation_28 (Activation)   (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 24, 24, 64)       

In [0]:
pruned_keras_file = path + 'pruned_mobilenet.h5'
print('Saving pruned model to: ', pruned_keras_file)

# No need to save the optimizer with the graph for serving.
tf.keras.models.save_model(final_model, pruned_keras_file, include_optimizer=False)

Saving pruned model to:  drive/My Drive/interns/Kpit/pruned_mobilenet.h5


In [0]:
zip1 = path + 'raw_mobilenet.zip' 
with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(keras_file)
print("Size of the unpruned model before compression: %.2f Mb" % 
      (os.path.getsize(keras_file) / float(2**20)))
print("Size of the unpruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip1) / float(2**20)))

zip2 = path + 'pruned_mobilenet.zip' 
with zipfile.ZipFile(zip2, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(pruned_keras_file)
print("Size of the pruned model before compression: %.2f Mb" % 
      (os.path.getsize(pruned_keras_file) / float(2**20)))
print("Size of the pruned model after compression: %.2f Mb" % 
      (os.path.getsize(zip2) / float(2**20)))

Size of the unpruned model before compression: 12.57 Mb
Size of the unpruned model after compression: 11.49 Mb
Size of the pruned model before compression: 12.57 Mb
Size of the pruned model after compression: 2.67 Mb


In [0]:
tflite_model_file = path + 'sparse_mobilenet.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)
tflite_model = converter.convert()
with open(tflite_model_file, 'wb') as f:
  f.write(tflite_model)

W0710 19:52:47.943387 140356938360704 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [0]:
zip_tflite = path + 'pruned_mobilenet_tflite.zip'
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 12.23 Mb
Size of the tflite model after compression: 2.53 Mb


In [0]:
import numpy as np

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 48, 48, 1))
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))

  return float(num_correct) / float(total_seen)


print(eval_model(interpreter, x_test, y_test))

Accuracy after 1000 images: 0.489000
Accuracy after 2000 images: 0.497000
Accuracy after 3000 images: 0.505667
0.5093340763443857


In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(pruned_keras_file)

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

tflite_quant_model = converter.convert()

tflite_quant_model_file = path + 'sparse_mobilenet_quant.tflite'
with open(tflite_quant_model_file, 'wb') as f:
  f.write(tflite_quant_model)

W0710 19:55:55.936868 140356938360704 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [0]:
zip_tflite = path + 'pruned_quant_mobilenet_tflite.zip'
with zipfile.ZipFile(zip_tflite, 'w', compression=zipfile.ZIP_DEFLATED) as f:
  f.write(tflite_quant_model_file)
print("Size of the tflite model before compression: %.2f Mb" 
      % (os.path.getsize(tflite_quant_model_file) / float(2**20)))
print("Size of the tflite model after compression: %.2f Mb" 
      % (os.path.getsize(zip_tflite) / float(2**20)))

Size of the tflite model before compression: 3.10 Mb
Size of the tflite model after compression: 0.69 Mb


In [0]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

st = time.time()
print(eval_model(interpreter, x_test, y_test))
end = time.time()
print(end - st)

Accuracy after 1000 images: 0.487000
Accuracy after 2000 images: 0.492000
Accuracy after 3000 images: 0.501667
0.5054332683198662
20.64324688911438


In [0]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_quant_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

num_correct = 0
aa = time.time()
for img,label in zip(x_test, y_test):
    inp = img.reshape((1,48,48,1))
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    output = interpreter.get_tensor(output_index)
    if np.argmax(output) == np.argmax(label):
        num_correct = num_correct + 1
time_elapsed = time.time() - aa

print('total_images : ', len(x_test), 'accuracy : ' , float(num_correct)/len(x_test))
print('total time :', time_elapsed , 'time per image: ', time_elapsed/len(x_test), 'fps : ', len(x_test)/time_elapsed)
